In [1]:
#! mkdir /wsefs/pipeline/etadata

In [2]:
import rioxarray as rioxa
from glob import glob
from dask.distributed import Client, LocalCluster, Lock
import os

def main():
    # instantiate the client as the do-er
    # docs.das.org/en/latest/deploying-python.html
    cluster = LocalCluster()
    # client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
    client = Client(cluster)
    print(client)
    print(cluster)

    etf = '/wsefs/pipeline/babadata/etfca_2022102.tif'
    # refet_location = 's3://ws-in/ssebop/etr/Hobbins/etr_102.tif'
    etr = '/wsefs/pipeline/ndata/etr_raster.tif'
    output_location = '/wsefs/pipeline/etadata/eta_2022102.tif'

    # open the fill raster
    etf_ds = rioxa.open_rasterio(etf, chunks='auto').squeeze().drop(labels='band')

    # the refet needs to get virtualwarped
    refet_ds = rioxa.open_rasterio(etr, chunks='auto').squeeze().drop(labels='band')
    eta = etf_ds * refet_ds
    print(eta)
    eta.rio.to_raster(output_location,compress='LZW', tiled=True, lock=Lock('rio', client=client))


if __name__ == "__main__":
    main()

<Client: 'tcp://127.0.0.1:40505' processes=4 threads=16, memory=61.58 GiB>
LocalCluster(1b929acc, 'tcp://127.0.0.1:40505', workers=4, threads=16, memory=61.58 GiB)
<xarray.DataArray (y: 14500, x: 37200)>
dask.array<mul, shape=(14500, 37200), dtype=float32, chunksize=(5632, 5632), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 179.0 179.0 179.0
  * y            (y) float64 80.0 79.99 79.98 79.97 ... -59.93 -59.94 -59.95
    spatial_ref  int64 0
